In [ ]:
!pip install netron

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.4 MB/s eta 0:00:00


In [ ]:
import netron
import os

# Path to your model file
model_path = 'path_to_your_model.onnx'  # Replace with your model's path

# Start Netron server
netron_url = netron.start(model_path, port=8080)  # Specify a port if needed
print(f"Netron is running at {netron_url}")

In [ ]:
import tensorflow as tf

In [ ]:
def create_tiny_embedding_conv_model(is_training = True, dropout_rate=0.2):

  if is_training:
    dropout_rate = dropout_rate
  input_frequency_size = 49
  input_time_size = 40
  fingerprint_4d = tf.reshape((1960, ),
                              [-1, input_time_size, input_frequency_size, 1])

  first_filter_width = 8
  first_filter_height = 10
  first_filter_count = 8
  first_weights = tf.compat.v1.get_variable(
      name='first_weights',
      initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.01),
      shape=[first_filter_height, first_filter_width, 1, first_filter_count])
  first_bias = tf.compat.v1.get_variable(
      name='first_bias',
      initializer=tf.compat.v1.zeros_initializer,
      shape=[first_filter_count])
  first_conv_stride_x = 2
  first_conv_stride_y = 2

  first_conv = tf.nn.conv2d(
      input=fingerprint_4d, filters=first_weights,
      strides=[1, first_conv_stride_y, first_conv_stride_x, 1],
      padding='SAME') + first_bias
  first_relu = tf.nn.relu(first_conv)
  if is_training:
    first_dropout = tf.nn.dropout(first_relu, rate=dropout_rate)

  else:
    first_dropout = first_relu

  second_filter_width = 8
  second_filter_height = 10
  second_filter_count = 8
  second_weights = tf.compat.v1.get_variable(
      name='second_weights',
      initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.01),
      shape=[
          second_filter_height, second_filter_width, first_filter_count,
          second_filter_count
      ])
  second_bias = tf.compat.v1.get_variable(
      name='second_bias',
      initializer=tf.compat.v1.zeros_initializer,
      shape=[second_filter_count])
  second_conv_stride_x = 8
  second_conv_stride_y = 8
  second_conv = tf.nn.conv2d(
      input=first_dropout, filters=second_weights,
      strides=[1, second_conv_stride_y, second_conv_stride_x, 1],
      padding='SAME') + second_bias
  second_relu = tf.nn.relu(second_conv)
  if is_training:
    second_dropout = tf.nn.dropout(second_relu, rate=dropout_rate)
  else:
    second_dropout = second_relu

  second_dropout_shape = second_dropout.get_shape()
  second_dropout_output_width = second_dropout_shape[2]
  second_dropout_output_height = second_dropout_shape[1]
  second_dropout_element_count = int(second_dropout_output_width *
                                     second_dropout_output_height *
                                     second_filter_count)
  flattened_second_dropout = tf.reshape(second_dropout,
                                        [-1, second_dropout_element_count])
  label_count = 4
  final_fc_weights = tf.compat.v1.get_variable(
      name='final_fc_weights',
      initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.01),
      shape=[second_dropout_element_count, label_count])
  final_fc_bias = tf.compat.v1.get_variable(
      name='final_fc_bias',
      initializer=tf.compat.v1.zeros_initializer,
      shape=[label_count])
  final_fc = (
      tf.matmul(flattened_second_dropout, final_fc_weights) + final_fc_bias)
  if is_training:
    return final_fc, dropout_rate
  else:
    return final_fc

In [ ]:

def create_tiny_embedding_conv_model(is_training=True, dropout_rate=0.2):
    input_frequency_size = 40
    input_time_size = 49
    label_count = 4

    # Define the input
    input_layer = tf.keras.layers.Input(shape=(1960,), name='input')

    # Reshape the input to 4D
    fingerprint_4d = tf.keras.layers.Reshape((input_time_size, input_frequency_size, 1))(input_layer)

    # First convolutional layer
    first_conv = tf.keras.layers.Conv2D(
        filters=8,
        kernel_size=(10, 8),
        strides=(2, 2),
        padding='same',
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01),
        bias_initializer=tf.keras.initializers.Zeros(),
        name='first_conv'
    )(fingerprint_4d)
    first_relu = tf.keras.layers.ReLU(name='first_relu')(first_conv)
    first_dropout = tf.keras.layers.Dropout(rate=dropout_rate if is_training else 0.0, name='first_dropout')(first_relu)

    # Second convolutional layer
    second_conv = tf.keras.layers.Conv2D(
        filters=8,
        kernel_size=(10, 8),
        strides=(8, 8),
        padding='same',
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01),
        bias_initializer=tf.keras.initializers.Zeros(),
        name='second_conv'
    )(first_dropout)
    second_relu = tf.keras.layers.ReLU(name='second_relu')(second_conv)
    second_dropout = tf.keras.layers.Dropout(rate=dropout_rate if is_training else 0.0, name='second_dropout')(second_relu)

    # Flatten the output
    flattened = tf.keras.layers.Flatten(name='flatten')(second_dropout)

    # Fully connected layer
    final_fc = tf.keras.layers.Dense(
        units=label_count,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01),
        bias_initializer=tf.keras.initializers.Zeros(),
        activation=None,  # Logits output
        name='final_fc'
    )(flattened)

    # Define the model
    model = tf.keras.models.Model(inputs=input_layer, outputs=final_fc, name='tiny_embedding_conv_model')
    return model

# Create and summarize the model
model = create_tiny_embedding_conv_model(is_training=True, dropout_rate=0.2)
model.summary()


Model: "tiny_embedding_conv_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 1960)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_3 (Reshape)                  │ (None, 49, 40, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ first_conv (Conv2D)                  │ (None, 25, 20, 8)           │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ first_relu (ReLU)                    │ (None, 25, 20, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ first_dropout (Dropout)              │ (None, 25, 20, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ second_conv (Conv2D)                 │ (None, 4, 3, 8)             │           5,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ second_relu (ReLU)                   │ (None, 4, 3, 8)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ second_dropout (Dropout)             │ (None, 4, 3, 8)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 96)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ final_fc (Dense)                     │ (None, 4)                   │             388 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,164 (24.08 KB)

 Trainable params: 6,164 (24.08 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model = create_tiny_embedding_conv_model(is_training=True, dropout_rate = 0.2)
model.summary()

# You can then compile and train the model as usual:
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))


InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 1 values, but the requested shape has 1960 [Op:Reshape]

In [ ]:
import tensorflow as tf
from tensorflow.lite.python import schema_py_generated as schema

def get_tflite_operations(tflite_model_path):
    # Load the TFLite model file
    with open(tflite_model_path, "rb") as f:
        model_data = f.read()

    # Parse the TFLite model
    model = schema.Model.GetRootAsModel(model_data, 0)

    # Get the subgraph (usually only one in most models)
    subgraph = model.Subgraphs(0)
    operators = subgraph.OperatorsLength()

    # Extract all the operation codes used in the model
    op_codes = []
    for i in range(operators):
        op_code_index = subgraph.Operators(i).OpcodeIndex()
        op_code = model.OperatorCodes(op_code_index).BuiltinCode()
        op_codes.append(op_code)

    # Map operation codes to their names using BuiltinOperator.__dict__
    op_names = [key for key, value in schema.BuiltinOperator.__dict__.items() if value in op_codes]
    return set(op_names)  # Return unique operation names

# Path to your TFLite model
tflite_model_path = "qat_sequential_23.tflite"

# Get and print operations
operations = get_tflite_operations(tflite_model_path)
print("Operations used in the TFLite model:")
for op in operations:
    print(op)


Operations used in the TFLite model:
CONV_2D
SHAPE
PACK
SOFTMAX
QUANTIZE
RESHAPE
STRIDED_SLICE
DEQUANTIZE
FULLY_CONNECTED


In [ ]:
"""Library for converting .tflite, .bmp and .wav files to cc arrays."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import struct
import wave
import numpy as np

from PIL import Image


def generate_file(out_fname, array_name, array_type, array_contents, size):
  """Write an array of values to a CC or header file."""
  os.makedirs(os.path.dirname(out_fname), exist_ok=True)
  if out_fname.endswith('.cc'):
    out_cc_file = open(out_fname, 'w')
    out_cc_file.write('#include <cstdint>\n\n')
    out_cc_file.write('#include "{}"\n\n'.format(
        out_fname.split('genfiles/')[-1].replace('.cc', '.h')))
    out_cc_file.write('alignas(16) const {} {}[] = {{'.format(
        array_type, array_name))
    out_cc_file.write(array_contents)
    out_cc_file.write('};\n')
    out_cc_file.close()
  elif out_fname.endswith('.h'):
    out_hdr_file = open(out_fname, 'w')
    out_hdr_file.write('#include <cstdint>\n\n')
    out_hdr_file.write('constexpr unsigned int {}_size = {};\n'.format(
        array_name, str(size)))
    out_hdr_file.write('extern const {} {}[];\n'.format(
        array_type, array_name))
    out_hdr_file.close()
  else:
    raise ValueError('generated file must be end with .cc or .h')


def bytes_to_hexstring(buffer):
  """Convert a byte array to a hex string."""
  hex_values = [hex(buffer[i]) for i in range(len(buffer))]
  out_string = ','.join(hex_values)
  return out_string


def generate_array(input_fname):
  """Return array size and array of data from the input file."""
  if input_fname.endswith('.tflite'):
    with open(input_fname, 'rb') as input_file:
      buffer = input_file.read()
    size = len(buffer)
    out_string = bytes_to_hexstring(buffer)
    return [size, out_string]
  elif input_fname.endswith('.bmp'):
    img = Image.open(input_fname, mode='r')
    image_bytes = img.tobytes()
    size = len(image_bytes)
    out_string = bytes_to_hexstring(image_bytes)
    return [size, out_string]
  elif input_fname.endswith('.wav'):
    wav_file = wave.open(input_fname, mode='r')
    num_channels = wav_file.getnchannels()
    n_frames = wav_file.getnframes()
    frames = wav_file.readframes(n_frames)
    samples = struct.unpack('<%dh' % (num_channels * n_frames), frames)
    out_string = ','.join(map(str, samples))
    wav_file.close()
    return [wav_file.getnframes(), out_string]
  elif input_fname.endswith('.csv'):
    with open(input_fname, 'r') as input_file:
      # Assume one array per csv file.
      elements = input_file.readline()
      return [len(elements.split(',')), elements]
  elif input_fname.endswith('.npy'):
    data = np.float32(np.load(input_fname, allow_pickle=False))
    data_1d = data.flatten()
    out_string = ','.join([str(x) for x in data_1d])
    return [len(data_1d), out_string]

  else:
    raise ValueError('input file must be .tflite, .bmp, .wav or .csv')


def get_array_name(input_fname):
  # Normalize potential relative path to remove additional dot.
  abs_fname = os.path.abspath(input_fname)
  base_array_name = 'g_' + abs_fname.split('.')[-2].split('/')[-1]
  if input_fname.endswith('.tflite'):
    return [base_array_name + '_model_data', 'unsigned char']
  elif input_fname.endswith('.bmp'):
    return [base_array_name + '_image_data', 'unsigned char']
  elif input_fname.endswith('.wav'):
    return [base_array_name + '_audio_data', 'int16_t']
  elif input_fname.endswith('_int32.csv'):
    return [base_array_name + '_test_data', 'int32_t']
  elif input_fname.endswith('_int16.csv'):
    return [base_array_name + '_test_data', 'int16_t']
  elif input_fname.endswith('_int8.csv'):
    return [base_array_name + '_test_data', 'int8_t']
  elif input_fname.endswith('_float.csv'):
    return [base_array_name + '_test_data', 'float']
  elif input_fname.endswith('npy'):
    return [base_array_name + '_test_data', 'float']





In [ ]:
def main(output, inputs):
    """Create cc sources with c arrays with data from each .tflite or .bmp."""
    if output.endswith('.cc') or output.endswith('.h'):
        assert len(inputs) == 1
        size, cc_array = generate_array(inputs[0])
        generated_array_name, array_type = get_array_name(inputs[0])
        generate_file(output, generated_array_name, array_type, cc_array, size)
    else:
        # Deduplicate inputs to prevent duplicate generated files (ODR issue).
        for input_file in list(dict.fromkeys(inputs)):
            output_base_fname = os.path.join(output, os.path.splitext(input_file)[0])
            if input_file.endswith('.tflite'):
                output_base_fname += '_model_data'
            elif input_file.endswith('.bmp'):
                output_base_fname += '_image_data'
            elif input_file.endswith('.wav'):
                output_base_fname += '_audio_data'
            elif input_file.endswith('.csv'):
                output_base_fname += '_test_data'
            elif input_file.endswith('.npy'):
                output_base_fname += '_test_data'
            else:
                raise ValueError('Input file must be .tflite, .bmp, .wav, .npy, or .csv')

            output_cc_fname = output_base_fname + '.cc'
            print("Generated:", output_cc_fname)  # For visibility in Colab output
            output_hdr_fname = output_base_fname + '.h'
            size, cc_array = generate_array(input_file)
            generated_array_name, array_type = get_array_name(input_file)
            generate_file(output_cc_fname, generated_array_name, array_type, cc_array, size)
            generate_file(output_hdr_fname, generated_array_name, array_type, cc_array, size)


In [ ]:
# Example Usage in Colab:
# Replace 'output_dir' and 'input_files' with your desired paths and files.
output_dir = '/content/'
input_files = ['audio_preprocessor_float.tflite']

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Run the main function
main(output_dir, input_files)

Generated: /content/audio_preprocessor_float_model_data.cc


FileNotFoundError: [Errno 2] No such file or directory: 'audio_preprocessor_float.tflite'